In [221]:
import os
import json
import pandas as pd



In [222]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq


In [223]:
KEY=os.getenv("GROQ_API_KEY")


In [224]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [225]:
llm=ChatGroq(temperature=0.5, groq_api_key=KEY, model_name="llama3-70b-8192")

In [226]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000025774BFD710>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025774059550>, model_name='llama3-70b-8192', temperature=0.5, groq_api_key=SecretStr('**********'))

In [227]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2


In [228]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [229]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
Do not provide any extra text such as 'Here are the 5 multiple choice questions:'
"""

In [230]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )


In [231]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [232]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}
Check from an expert English Writer of the above quiz:
"""

In [233]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [234]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [235]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [236]:
file_path=r"D:/CODING/GEN AI/mcqgenerator/data.txt"

In [237]:
file_path

'D:/CODING/GEN AI/mcqgenerator/data.txt'

In [238]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [239]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]

From a theoretical 

In [240]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [241]:
NUMBER=10
SUBJECT="Machine Learning"
TONE="complex"


In [242]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of 

In [243]:
response

{'text': "Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]\n\nFro

In [244]:
quiz=response.get("quiz")

print(quiz)

{"1": {"mcq": "Machine learning is a field of study in artificial intelligence concerned with the development and study of which type of algorithms?", "options": {"a": "Statistical algorithms", "b": "Deterministic algorithms", "c": "Heuristic algorithms", "d": "Dynamic algorithms"}, "correct": "a"}, 
"2": {"mcq": "Which type of neural networks have been able to surpass many previous approaches in performance?", "options": {"a": "Artificial neural networks", "b": "Biological neural networks", "c": "Convolutional neural networks", "d": "Recurrent neural networks"}, "correct": "a"}, 
"3": {"mcq": "What is the term used to describe the application of machine learning to business problems?", "options": {"a": "Predictive modeling", "b": "Predictive analytics", "c": "Business intelligence", "d": "Data mining"}, "correct": "b"}, 
"4": {"mcq": "Which field of study focuses on exploratory data analysis through unsupervised learning?", "options": {"a": "Machine learning", "b": "Data mining", "c":

In [245]:
abc=json.loads(quiz)

In [246]:
quiz_table_data = []
for key, value in abc.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [247]:
quiz_table_data

[{'MCQ': 'Machine learning is a field of study in artificial intelligence concerned with the development and study of which type of algorithms?',
  'Choices': 'a: Statistical algorithms | b: Deterministic algorithms | c: Heuristic algorithms | d: Dynamic algorithms',
  'Correct': 'a'},
 {'MCQ': 'Which type of neural networks have been able to surpass many previous approaches in performance?',
  'Choices': 'a: Artificial neural networks | b: Biological neural networks | c: Convolutional neural networks | d: Recurrent neural networks',
  'Correct': 'a'},
 {'MCQ': 'What is the term used to describe the application of machine learning to business problems?',
  'Choices': 'a: Predictive modeling | b: Predictive analytics | c: Business intelligence | d: Data mining',
  'Correct': 'b'},
 {'MCQ': 'Which field of study focuses on exploratory data analysis through unsupervised learning?',
  'Choices': 'a: Machine learning | b: Data mining | c: Computational statistics | d: Mathematical optimizat

In [248]:
quiz=pd.DataFrame(quiz_table_data)

In [249]:
quiz.to_csv("machinelearning.csv",index=False)

In [250]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'08_19_2024_04_56_55'